# getting


# 新しいセクション

## modules

In [ ]:
!pip install optuna xfeat japanize-matplotlib >> /dev/null

In [ ]:
import re

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
sns.set()

from glob import glob
from functools import partial
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xfeat import SelectCategorical, LabelEncoder, LambdaEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

## data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
paths = glob('/content/drive/MyDrive/データ/2022_12_Nishika_estate/data/train/*')
train_dfs = []
for path in paths:
  train_df = pd.read_csv(path)
  train_dfs.append(train_df)
train_df = pd.concat(train_dfs)
train_df.reset_index(drop=True, inplace = True)
test_df = pd.read_csv('/content/drive/MyDrive/データ/2022_12_Nishika_estate/data/test.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## formula

In [ ]:
def normalize_moyori(moyori):
    if moyori == moyori:
        if moyori == '30分?60分':
            moyori = 45
        elif moyori == '1H?1H30':
            moyori = 75
        elif moyori == '1H30?2H':
            moyori = 105
        elif moyori == '2H?':
            moyori = 120
        moyori = int(moyori)
    return moyori

In [ ]:
def normalize_area(area):
    if area == area:
        area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
    return area

In [ ]:
def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

# adjusting data

## 整形

In [ ]:
train_df.columns

Index(['ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造',
       '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log'],
      dtype='object')

In [ ]:
df = train_df[['ID', '種類', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）',  '建築年', '建物の構造', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引価格（総額）_log']]

In [ ]:
ID = 'ID'
TARGET = '取引価格（総額）_log'
CODE = '市区町村コード'
STATION = '最寄駅：名称'
DIMENTION = '面積（㎡）'
KEMPEI = '建ぺい率（％）'
FLOOR = '容積率（％）'
# 短縮ダイヤル

## quanitify


### 1.単純なやつ

In [ ]:
df['建築年'] = df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
df['面積（㎡）'] = df['面積（㎡）'].apply(lambda x: normalize_area(x))
df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))

<ipython-input-12-1f0f67c6feb2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['建築年'] = df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
<ipython-input-12-1f0f67c6feb2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['面積（㎡）'] = df['面積（㎡）'].apply(lambda x: normalize_area(x))
<ipython-input-12-1f0f67c6feb2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [ ]:
df['改装'].unique()

array(['未改装', nan, '改装済'], dtype=object)

In [ ]:
df['改装'] = df['改装'].map(lambda x: 1 if x == '改装済' else 0)

<ipython-input-14-8f570304d8d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['改装'] = df['改装'].map(lambda x: 1 if x == '改装済' else 0)


# try 全部one hot

### 間取り いじるのはdf1_1
部屋の数だけ取り出す


In [ ]:
print(df['間取り'].unique())
print(df['間取り'].unique().size)

['１Ｋ' '４ＬＤＫ' '３ＬＤＫ' '１ＤＫ' '２ＬＤＫ' '２ＤＫ' '１ＬＤＫ' nan '１Ｒ' '３ＤＫ' '２ＬＤＫ＋Ｓ'
 'オープンフロア' '２Ｋ' '３Ｋ' '２ＤＫ＋Ｓ' '４ＤＫ' '５ＬＤＫ' '１ＬＤＫ＋Ｓ' '４Ｋ' '３ＬＤＫ＋Ｓ' '４ＬＤＫ＋Ｓ'
 '１ＤＫ＋Ｓ' '３ＬＫ' '１Ｌ' '５ＬＤＫ＋Ｓ' '３ＬＤ' '２ＬＫ' 'メゾネット' '２Ｌ' '３ＤＫ＋Ｓ' '３Ｋ＋Ｓ' '５ＤＫ'
 '４Ｌ＋Ｋ' '１ＤＫ＋Ｋ' '１ＬＫ' '２ＬＤＫ＋Ｋ' '２Ｋ＋Ｓ' '２ＬＫ＋Ｓ' '６ＬＤＫ' '４ＤＫ＋Ｓ' '４ＬＤＫ＋Ｋ'
 '２ＬＤ＋Ｓ' '１Ｒ＋Ｓ' 'スタジオ' '１Ｌ＋Ｓ' '３ＬＤ＋Ｓ' '３ＬＤＫ＋Ｋ' '１Ｋ＋Ｓ' '１ＬＫ＋Ｓ' '７ＬＤＫ' '６ＤＫ'
 '１ＬＤ＋Ｓ' '２Ｌ＋Ｓ' '２Ｄ' '２ＬＤ' '６ＬＤＫ＋Ｓ' '４Ｄ' '３Ｄ' '５Ｋ' '５ＬＤＫ＋Ｋ' '８ＬＤＫ'
 '８ＬＤＫ＋Ｓ' '７ＬＤＫ＋Ｓ' '４Ｌ' '３ＬＫ＋Ｓ' '７ＤＫ']
66


In [ ]:
'１' in '１Ｋ'

True

In [ ]:
def define_madori(x):
  if '１' in str(x) :
    y = 1
  elif '２' in str(x):
    y = 2
  elif '３' in str(x):
    y = 3
  elif '４' in str(x):
    y = 4
  elif '５' in str(x):
    y = 5
  elif '６' in str(x):
    y = 6
  elif '７' in str(x):
    y = 7
  elif '８' in str(x):
    y = 8
  else:
    y = 0
  return y

In [ ]:
define_madori('３ＬＤ＋Ｓ')

3

In [ ]:
df['間取り'] = df['間取り'].apply(lambda x: define_madori(x))

<ipython-input-19-38b346abfe35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['間取り'] = df['間取り'].apply(lambda x: define_madori(x))


In [ ]:
df.head()

,ID,種類,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引価格（総額）_log
0,3015552,中古マンション等,3201,岩手県,盛岡市,東山,盛岡,45.0,1,25,NaN,ＲＣ,第２種中高層住居専用地域,60.0,200.0,2015年第2四半期,0,6.301030
1,3023488,中古マンション等,3201,岩手県,盛岡市,本町通,上盛岡,1.0,4,90,1999.0,ＳＲＣ,近隣商業地域,80.0,200.0,2008年第1四半期,0,7.380211
2,3001812,中古マンション等,3201,岩手県,盛岡市,中央通,盛岡,15.0,4,90,2015.0,ＲＣ,商業地域,80.0,600.0,2019年第4四半期,0,7.568202
3,3009546,中古マンション等,3201,岩手県,盛岡市,大沢川原,盛岡,11.0,3,75,NaN,ＲＣ,商業地域,80.0,400.0,2010年第4四半期,0,7.278754
4,3018911,中古マンション等,3201,岩手県,盛岡市,材木町,盛岡,10.0,3,65,1985.0,ＳＲＣ,商業地域,80.0,400.0,2008年第4四半期,0,6.982271


## 1_2 種類

In [ ]:
df['種類'].unique()

array(['中古マンション等'], dtype=object)

これしかないのでのでcut

In [ ]:
df = df.drop('種類', axis = 1)

## 1_3 都市計画

In [ ]:
df['都市計画']

0         第２種中高層住居専用地域
1               近隣商業地域
2                 商業地域
3                 商業地域
4                 商業地域
              ...     
736705         第２種住居地域
736706            商業地域
736707          近隣商業地域
736708    第２種中高層住居専用地域
736709         第２種住居地域
Name: 都市計画, Length: 736710, dtype: object

In [ ]:
df['都市計画'].unique()

array(['第２種中高層住居専用地域', '近隣商業地域', '商業地域', '第１種住居地域', nan, '第２種住居地域',
       '第１種中高層住居専用地域', '都市計画区域外', '準工業地域', '市街化区域及び市街化調整区域外の都市計画区域',
       '第１種低層住居専用地域', '準住居地域', '第２種低層住居専用地域', '工業地域', '工業専用地域', '市街化調整区域',
       '準都市計画区域'], dtype=object)

In [ ]:
def toshikeikaku(x):
  if '中高層住居専用地域' in str(x):
    y = '高層'
  elif '商業地域' in str(x):
    y = '商業'
  elif '住居地域' in str(x):
    y = '住居'
  elif '工業' in str(x):
    y = '工業'
  else:
    y = 'その他'
  return y

In [ ]:
df['都市計画']=df['都市計画'].apply(lambda x : toshikeikaku(x))

In [ ]:
df['都市計画']

0         高層
1         商業
2         商業
3         商業
4         商業
          ..
736705    住居
736706    商業
736707    商業
736708    高層
736709    住居
Name: 都市計画, Length: 736710, dtype: object

In [ ]:
df_dummies=pd.get_dummies(df['都市計画'])
#mushroom_dummy = pd.get_dummies(mushroom[['gill_color', 'gill_attachment', 'odor', 'cap_color']])

In [ ]:
df = pd.concat([df, df_dummies] , axis = 1)

In [ ]:
df.head()

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,...,建ぺい率（％）,容積率（％）,取引時点,改装,取引価格（総額）_log,その他,住居,商業,工業,高層
0,3015552,3201,岩手県,盛岡市,東山,盛岡,45.0,1,25,NaN,...,60.0,200.0,2015年第2四半期,0,6.301030,0,0,0,0,1
1,3023488,3201,岩手県,盛岡市,本町通,上盛岡,1.0,4,90,1999.0,...,80.0,200.0,2008年第1四半期,0,7.380211,0,0,1,0,0
2,3001812,3201,岩手県,盛岡市,中央通,盛岡,15.0,4,90,2015.0,...,80.0,600.0,2019年第4四半期,0,7.568202,0,0,1,0,0
3,3009546,3201,岩手県,盛岡市,大沢川原,盛岡,11.0,3,75,NaN,...,80.0,400.0,2010年第4四半期,0,7.278754,0,0,1,0,0
4,3018911,3201,岩手県,盛岡市,材木町,盛岡,10.0,3,65,1985.0,...,80.0,400.0,2008年第4四半期,0,6.982271,0,0,1,0,0


## 1_3' 取引時点も落としちゃう

In [ ]:
df = df.drop(['取引時点', '都市計画'], axis = 1)

In [ ]:
df.head()

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,建ぺい率（％）,容積率（％）,改装,取引価格（総額）_log,その他,住居,商業,工業,高層
0,3015552,3201,岩手県,盛岡市,東山,盛岡,45.0,1,25,NaN,ＲＣ,60.0,200.0,0,6.301030,0,0,0,0,1
1,3023488,3201,岩手県,盛岡市,本町通,上盛岡,1.0,4,90,1999.0,ＳＲＣ,80.0,200.0,0,7.380211,0,0,1,0,0
2,3001812,3201,岩手県,盛岡市,中央通,盛岡,15.0,4,90,2015.0,ＲＣ,80.0,600.0,0,7.568202,0,0,1,0,0
3,3009546,3201,岩手県,盛岡市,大沢川原,盛岡,11.0,3,75,NaN,ＲＣ,80.0,400.0,0,7.278754,0,0,1,0,0
4,3018911,3201,岩手県,盛岡市,材木町,盛岡,10.0,3,65,1985.0,ＳＲＣ,80.0,400.0,0,6.982271,0,0,1,0,0


In [ ]:
df1_4 = df.copy()

## 1_4 構造

In [ ]:
df1_4['建物の構造'].unique()

array(['ＲＣ', 'ＳＲＣ', nan, '鉄骨造', 'ＲＣ、鉄骨造', 'ブロック造', 'ＳＲＣ、ＲＣ', '木造',
       'ＲＣ、ブロック造', '軽量鉄骨造', 'ＲＣ、木造', 'ＳＲＣ、鉄骨造', 'ＳＲＣ、ＲＣ、鉄骨造'],
      dtype=object)

In [ ]:
def structure(x):
  if 'ＳＲＣ' in str(x):
    y = 'SRC'
  elif 'ＲＣ' in str(x):
    y = 'RC'
  else :
    y = 'other'
  return y

In [ ]:
df1_4['建物の構造']=df1_4['建物の構造'].apply(lambda x : structure(x))

In [ ]:
df1_4_dummies = pd.get_dummies(df1_4['建物の構造'])

In [ ]:
df1_4 = pd.concat([df1_4, df1_4_dummies], axis = 1)

In [ ]:
df1_4 = df1_4.drop(['建物の構造'], axis=1)

In [ ]:
pd.get_option("display.max_columns")
pd.set_option('display.max_columns', 50)
df1_4.head()

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,改装,取引価格（総額）_log,その他,住居,商業,工業,高層,RC,SRC,other
0,3015552,3201,岩手県,盛岡市,東山,盛岡,45.0,1,25,NaN,60.0,200.0,0,6.301030,0,0,0,0,1,1,0,0
1,3023488,3201,岩手県,盛岡市,本町通,上盛岡,1.0,4,90,1999.0,80.0,200.0,0,7.380211,0,0,1,0,0,0,1,0
2,3001812,3201,岩手県,盛岡市,中央通,盛岡,15.0,4,90,2015.0,80.0,600.0,0,7.568202,0,0,1,0,0,1,0,0
3,3009546,3201,岩手県,盛岡市,大沢川原,盛岡,11.0,3,75,NaN,80.0,400.0,0,7.278754,0,0,1,0,0,1,0,0
4,3018911,3201,岩手県,盛岡市,材木町,盛岡,10.0,3,65,1985.0,80.0,400.0,0,6.982271,0,0,1,0,0,0,1,0


## 1_5 建築年, 最寄りまでの距離、建ぺい率を平均で埋める

In [ ]:
df1_5 = df1_4.copy()

In [ ]:
df1_5['建築年'] = df1_5['建築年'].fillna(df1_5['建築年'].mean())

In [ ]:
df1_5['建築年'].isna().sum()
#df['column name'].isna().sum()

0

In [ ]:
df1_5['最寄駅：距離（分）'] = df1_5['最寄駅：距離（分）'].fillna(df1_5['最寄駅：距離（分）'].mean())

In [ ]:
df1_5[KEMPEI] = df1_5[KEMPEI].fillna(df1_5[KEMPEI].mean())

In [ ]:
df1_5[FLOOR] = df1_5[FLOOR].fillna(df1_5[FLOOR].mean())

## 1_6 場所 市区町村でdummies


In [ ]:
df1_6 = df1_5.copy()

In [ ]:
df1_6_dummies = pd.get_dummies(df1_6['市区町村名'])
df1_6 = df1_6.drop(['市区町村コード', '都道府県名', '地区名', STATION, '市区町村名'], axis =1)

In [ ]:
df1_6 = pd.concat([df1_6, df1_6_dummies], axis=1)

# テストデータいじり

In [ ]:
testdf = test_df[['ID', '種類', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）',  '建築年', '建物の構造', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装']]
testdf['建築年'] = testdf['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
testdf['面積（㎡）'] = testdf['面積（㎡）'].apply(lambda x: normalize_area(x))
testdf['最寄駅：距離（分）'] = testdf['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))

<ipython-input-50-b214b16d3423>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf['建築年'] = testdf['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
<ipython-input-50-b214b16d3423>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf['面積（㎡）'] = testdf['面積（㎡）'].apply(lambda x: normalize_area(x))
<ipython-input-50-b214b16d3423>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [ ]:
testdf['改装'] = testdf['改装'].map(lambda x: 1 if x == '改装済' else 0)

<ipython-input-51-bfee689b5651>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf['改装'] = testdf['改装'].map(lambda x: 1 if x == '改装済' else 0)


In [ ]:
testdf['間取り'] = testdf['間取り'].apply(lambda x: define_madori(x))

<ipython-input-52-d54616887bef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf['間取り'] = testdf['間取り'].apply(lambda x: define_madori(x))


In [ ]:
testdf = testdf.drop('種類', axis = 1)

In [ ]:
testdf = testdf.drop(['取引時点', '都市計画'], axis = 1)

In [ ]:
testdf['建物の構造']=testdf['建物の構造'].apply(lambda x : structure(x))
test_dummies = pd.get_dummies(testdf['建物の構造'])
testdf = pd.concat([testdf, test_dummies], axis = 1)
testdf = testdf.drop('建物の構造', axis=1)

In [ ]:
testdf['建築年'] = testdf['建築年'].fillna(df1_4['建築年'].mean())
testdf['最寄駅：距離（分）']= testdf['最寄駅：距離（分）'].fillna(df1_4['最寄駅：距離（分）'].mean())
testdf[KEMPEI] = testdf[KEMPEI].fillna(df1_4[KEMPEI].mean())

In [ ]:
testdf_dummies = pd.get_dummies(testdf['市区町村名'])
testdf = testdf.drop(['市区町村コード', '都道府県名', '地区名', STATION, '市区町村名'], axis =1)

In [ ]:
testdf = pd.concat([testdf, testdf_dummies], axis=1)

In [ ]:
testdf[FLOOR] = testdf[FLOOR].fillna(df1_4[FLOOR].mean())

# 回帰

In [ ]:
df2 = df1_6.copy()

In [ ]:
test2 = testdf.copy()

In [ ]:
df2.head()

,ID,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,改装,取引価格（総額）_log,その他,住居,商業,工業,高層,RC,SRC,other,あきる野市,あま市,いわき市,うるま市,さいたま市中央区,さいたま市北区,さいたま市南区,さいたま市大宮区,...,静岡市清水区,静岡市葵区,静岡市駿河区,額田郡幸田町,飯塚市,飯田市,飯能市,香芝市,高山市,高岡市,高崎市,高座郡寒川町,高松市,高槻市,高浜市,高知市,高石市,鳥取市,鴻巣市,鶴ケ島市,鶴岡市,鹿児島市,鹿屋市,鹿沼市,龍ケ崎市
0,3015552,45.0,1,25,1995.433082,60.0,200.0,0,6.301030,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3023488,1.0,4,90,1999.000000,80.0,200.0,0,7.380211,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3001812,15.0,4,90,2015.000000,80.0,600.0,0,7.568202,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3009546,11.0,3,75,1995.433082,80.0,400.0,0,7.278754,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3018911,10.0,3,65,1985.000000,80.0,400.0,0,6.982271,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
testdf.head()

,ID,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,改装,RC,SRC,other,あま市,いわき市,さいたま市中央区,さいたま市北区,さいたま市南区,さいたま市大宮区,さいたま市岩槻区,さいたま市桜区,さいたま市浦和区,さいたま市緑区,さいたま市西区,さいたま市見沼区,つくばみらい市,つくば市,...,霧島市,青梅市,青森市,静岡市清水区,静岡市葵区,静岡市駿河区,額田郡幸田町,飯塚市,飯田市,飯能市,香芝市,高岡市,高崎市,高座郡寒川町,高松市,高槻市,高浜市,高知市,高石市,鳥取市,鴻巣市,鶴ケ島市,鹿児島市,鹿沼市,龍ケ崎市
0,1000118,2.0,2,50,1989.0,80.0,400.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1000121,2.0,0,15,1989.0,80.0,400.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1000122,2.0,2,50,1989.0,80.0,400.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1000124,2.0,1,45,1991.0,80.0,400.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1000125,2.0,3,70,1986.0,80.0,400.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_train = df2.drop(['ID', TARGET], axis = 1)

In [ ]:
y_train = df2[TARGET]

In [ ]:
X_test = testdf.drop('ID', axis=1)

## 2_1 リッジ回帰

In [ ]:
# データ分割（訓練データとテストデータ）のためのインポート
from sklearn.model_selection import train_test_split

# 重回帰のモデル構築のためのインポート
from sklearn.linear_model import LinearRegression

In [ ]:
# リッジ回帰用のクラス
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

In [ ]:
# モデルの構築と評価
linear = LinearRegression()
ridge = Ridge(random_state=0)

In [ ]:
for model in [linear, ridge]:
    model.fit(X_train, y_train)
    print('{}(train):{:.6f}'.format(model.__class__.__name__ , model.score(X_train, y_train)))
#    print('{}(test):{:.6f}'.format(model.__class__.__name__ , model.score(X_test, y_test)))

LinearRegression(train):0.728797
Ridge(train):0.728775


In [ ]:
y_test = model.predict(X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- あきる野市
- うるま市
- その他
- 一関市
- 三重郡川越町
- ...

  warnings.warn(message, FutureWarning)


ValueError: ignored